## Spotify part - getting the related artists

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd
from IPython.display import clear_output
from IPython.display import display

In [2]:
# Establish a session and instantiate it
client_credentials_manager = SpotifyClientCredentials(client_id='ccae2e7a3a0f48088d24563289eb71d0',
                                                      client_secret='0204fbff5f2c438a816b7c01ede9d9ce')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
artist_name = 'Jorge Drexler'
related_artists = []
result = sp.search(q='artist:' + artist_name, type='artist')
try:
    name = result['artists']['items'][0]['name']
    uri = result['artists']['items'][0]['uri']

    related = sp.artist_related_artists(uri)
    print('Related artists for', name)
    for artist in related['artists']:
        related_artists.append(artist['name'])
    print(related_artists)
except:
    print("Error!")

SpotifyOauthError: Bad Request

Fitting how the artists appear in Genius' API

In [16]:
related_artists[2] = 'Fito Páez'
related_artists[-3] = 'Seru Giran'
related_artists[12] = 'Sui Generis (AR)'

## Genius part - getting the lyrics for each

In [4]:
import requests
import json
import requests
from bs4 import BeautifulSoup
import re

CLIENT_ACCESS_TOKEN = "lTv0ZCyw1mkXQ7_9bZ9GmEdc9vPh2_VQzhFCpERSOYiFA1fXjEVrI8BY-D9_k6E2"
BASE_URI = "https://api.genius.com"

In [25]:
#from: https://gist.github.com/imdkm/a60247b59ff1881fa4bb8846a9b44c96
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URI, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

artists_by_ids = {}
for artist_name in related_artists:
    print("searching " + artist_name + "'s artist id. \n")

    # find artist id from given data.
    find_id = _get("search", {'q': artist_name})
    for hit in find_id["response"]["hits"]:
       if hit["result"]["primary_artist"]["name"] == artist_name:
           artist_id = hit["result"]["primary_artist"]["id"]
           break
    
    artists_by_ids[artist_id] = artist_name
    print("-> " + artist_name + "'s id is " + str(artist_id) + "\n")

searching Kevin Johansen's artist id. 

-> Kevin Johansen's id is 353594

searching Pedro Guerra's artist id. 

-> Pedro Guerra's id is 356298

searching Fito Páez's artist id. 

-> Fito Páez's id is 356209

searching Luis Alberto Spinetta's artist id. 

-> Luis Alberto Spinetta's id is 344478

searching Pedro Aznar's artist id. 

-> Pedro Aznar's id is 344705

searching Ismael Serrano's artist id. 

-> Ismael Serrano's id is 55590

searching Martín Buscaglia's artist id. 

-> Martín Buscaglia's id is 381572

searching Ana Prada's artist id. 

-> Ana Prada's id is 458438

searching Mercedes Sosa's artist id. 

-> Mercedes Sosa's id is 74374

searching Silvio Rodríguez's artist id. 

-> Silvio Rodríguez's id is 154366

searching Joaquín Sabina's artist id. 

-> Joaquín Sabina's id is 315233

searching Fabiana Cantilo's artist id. 

-> Fabiana Cantilo's id is 352645

searching Sui Generis (AR)'s artist id. 

-> Sui Generis (AR)'s id is 1011974

searching Luis Eduardo Aute's artist id. 



In [32]:
print(len(artists_by_ids))
print(len(set(list(artists_by_ids.keys()))))

20
20


In [23]:
def get_artist_songs(artist_id):

    current_page = 1
    next_page = True
    songs = []

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}
        data = _get(path=path, params=params)
        page_songs = data['response']['songs']
        if page_songs:
            songs += page_songs
            current_page += 1
        else:
            next_page = False

    return songs

def scrape_lyrics(url):

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    lyrics = soup.find("div", {"class": "lyrics"})
    lyrics = cleanhtml(str(lyrics))
    return lyrics

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext



In [84]:
##### COMPLETE THIS LIST!
#artists_downloaded_already = ['Kevin Johansen', 'Pedro Guerra', 'Fito Páez']
### OR
df = pd.read_csv('data/related_raw_v1.csv', sep=';')
artists_downloaded_already = df.artist.unique().tolist()

In [82]:
with open('data/related_raw_v1.csv', 'a') as f:
    for an_id in artists_by_ids.keys():
        if artists_by_ids[an_id] in artists_downloaded_already:
            continue
        songs = get_artist_songs(an_id)
        songs = [song for song in songs if song['primary_artist']['id'] == an_id]

        for idx, song in enumerate(songs):
            clear_output(wait=True)
            print('Looking for %s with id %s' % (str(artists_by_ids[an_id]),str(an_id)))
            print('Song %s/%s' % (str(idx),str(len(songs))))
            url = song['url']
            lyrics = scrape_lyrics(url)
            song['lyrics'] = lyrics
            song['artist'] = artists_by_ids[an_id]
        df = pd.read_json(json.dumps(songs))
        df.to_csv(f, header=False, sep=';', index=False)
        artists_downloaded_already.append(artists_by_ids[an_id])
        display(df.tail(10))

Looking for Onda Vaga with id 365262
Song 52/53


,annotation_count,api_path,artist,full_title,header_image_thumbnail_url,header_image_url,id,lyrics,lyrics_owner_id,lyrics_state,path,primary_artist,pyongs_count,song_art_image_thumbnail_url,stats,title,title_with_featured,url
43,0,/songs/3322567,Onda Vaga,Ritmo Al Día by Onda Vaga,https://images.genius.com/368d40ea4897c55700e3...,https://images.genius.com/368d40ea4897c55700e3...,3322567,\n\n[Verso 1]\nLe puso ritmo al día\nComo si n...,3565002,complete,/Onda-vaga-ritmo-al-dia-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/368d40ea4897c55700e3...,"{'hot': False, 'unreviewed_annotations': 0}",Ritmo Al Día,Ritmo Al Día,https://genius.com/Onda-vaga-ritmo-al-dia-lyrics
44,1,/songs/2099395,Onda Vaga,Sentido la vida by Onda Vaga,https://images.genius.com/48072e4e750c8883ac23...,https://images.genius.com/48072e4e750c8883ac23...,2099395,\n\n¿Qué sentido la vida?\n¿Qué sentida mi vid...,1549345,complete,/Onda-vaga-sentido-la-vida-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/48072e4e750c8883ac23...,"{'hot': False, 'unreviewed_annotations': 0}",Sentido la vida,Sentido la vida,https://genius.com/Onda-vaga-sentido-la-vida-l...
45,1,/songs/1375541,Onda Vaga,Sequía De Amor by Onda Vaga,https://images.genius.com/d57fc63ea1de4ed8d07c...,https://images.genius.com/d57fc63ea1de4ed8d07c...,1375541,"\n\nDesde arriba a abajo, desde izquierda a de...",1549345,complete,/Onda-vaga-sequia-de-amor-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/d57fc63ea1de4ed8d07c...,"{'hot': False, 'unreviewed_annotations': 0}",Sequía De Amor,Sequía De Amor,https://genius.com/Onda-vaga-sequia-de-amor-ly...
46,0,/songs/3322514,Onda Vaga,Será Que Estás Ahí by Onda Vaga,https://images.genius.com/368d40ea4897c55700e3...,https://images.genius.com/368d40ea4897c55700e3...,3322514,"\n\n[Verso 1]\nSerá que estas ahí, todo el tie...",3565002,complete,/Onda-vaga-sera-que-estas-ahi-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/368d40ea4897c55700e3...,"{'hot': False, 'unreviewed_annotations': 0}",Será Que Estás Ahí,Será Que Estás Ahí,https://genius.com/Onda-vaga-sera-que-estas-ah...
47,1,/songs/2055101,Onda Vaga,Susanitas by Onda Vaga,https://images.genius.com/5e39900825c8060124a6...,https://images.genius.com/5e39900825c8060124a6...,2055101,"\n\nAia Susanita, tu cuerpo es lo que yo busca...",1549345,complete,/Onda-vaga-susanitas-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/5e39900825c8060124a6...,"{'hot': False, 'unreviewed_annotations': 0}",Susanitas,Susanitas,https://genius.com/Onda-vaga-susanitas-lyrics
48,1,/songs/2057496,Onda Vaga,Tataralí by Onda Vaga,https://images.genius.com/5e39900825c8060124a6...,https://images.genius.com/5e39900825c8060124a6...,2057496,\n\nHoy salí a caminar\nY me puse a cantar\nPo...,1549345,complete,/Onda-vaga-tatarali-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/5e39900825c8060124a6...,"{'hot': False, 'unreviewed_annotations': 0}",Tataralí,Tataralí,https://genius.com/Onda-vaga-tatarali-lyrics
49,1,/songs/1439144,Onda Vaga,Te Quiero by Onda Vaga,https://images.genius.com/1d6223358161d9c5c758...,https://images.genius.com/1d6223358161d9c5c758...,1439144,"\n\nTe quiero, pensé\nTe quiero, sos muy linda...",1549345,complete,/Onda-vaga-te-quiero-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/1d6223358161d9c5c758...,"{'hot': False, 'unreviewed_annotations': 0}",Te Quiero,Te Quiero,https://genius.com/Onda-vaga-te-quiero-lyrics
50,1,/songs/948505,Onda Vaga,Va Al Oeste by Onda Vaga,https://images.genius.com/1d6223358161d9c5c758...,https://images.genius.com/1d6223358161d9c5c758...,948505,\n\nNo me importa si el viento va al oeste\nO ...,1549345,complete,/Onda-vaga-va-al-oeste-lyrics,"{'api_path': '/artists/365262', 'header_image_...",NaN,https://images.genius.com/1d6223

In [86]:
df = pd.read_csv('data/related_raw_v1.csv', sep=';')
df.sample(10)

,annotation_count,api_path,artist,full_title,header_image_thumbnail_url,header_image_url,id,lyrics,lyrics_owner_id,lyrics_state,path,primary_artist,pyongs_count,song_art_image_thumbnail_url,stats,title,title_with_featured,url
1263,1,/songs/1329370,Mercedes Sosa,Venas abiertas by Mercedes Sosa,https://images.genius.com/0651a7e861ef11803001...,https://images.genius.com/0651a7e861ef11803001...,1329370,\n\nPor la blanda arena que lame el mar\nSu pe...,1549345,complete,/Mercedes-sosa-venas-abiertas-lyrics,"{'api_path': '/artists/74374', 'header_image_u...",NaN,https://images.genius.com/0651a7e861ef11803001...,"{'hot': False, 'unreviewed_annotations': 0}",Venas abiertas,Venas abiertas,https://genius.com/Mercedes-sosa-venas-abierta...
1464,1,/songs/1031730,Silvio Rodríguez,Martianos by Silvio Rodríguez,https://images.genius.com/73517ad2dd94f1cbbc34...,https://images.genius.com/73517ad2dd94f1cbbc34...,1031730,\n\nYo soy un grano de arena\nUna hoja más en ...,1549345,complete,/Silvio-rodriguez-martianos-lyrics,"{'api_path': '/artists/154366', 'header_image_...",NaN,https://images.genius.com/73517ad2dd94f1cbbc34...,"{'hot': False, 'unreviewed_annotations': 0}",Martianos,Martianos,https://genius.com/Silvio-rodriguez-martianos-...
1404,1,/songs/1468404,Silvio Rodríguez,Eva by Silvio Rodríguez,https://images.genius.com/ecc3a5d378cdf7eed0b4...,https://images.genius.com/ecc3a5d378cdf7eed0b4...,1468404,\n\nEva no quiere ser para Adán\nLa paridora p...,1549345,complete,/Silvio-rodriguez-eva-lyrics,"{'api_path': '/artists/154366', 'header_image_...",NaN,https://images.genius.com/ecc3a5d378cdf7eed0b4...,"{'hot': False, 'unreviewed_annotations': 0}",Eva,Eva,https://genius.com/Silvio-rodriguez-eva-lyrics
1455,1,/songs/1034625,Silvio Rodríguez,Llegué Por San Antonio De Los Baños by Silvio ...,https://images.genius.com/ecc3a5d378cdf7eed0b4...,https://images.genius.com/ecc3a5d378cdf7eed0b4...,1034625,\n\nLlegué por San Antonio de los Baños\nCuand...,1549345,complete,/Silvio-rodriguez-llegue-por-san-antonio-de-lo...,"{'api_path': '/artists/154366', 'header_image_...",NaN,https://images.genius.com/ecc3a5d378cdf7eed0b4...,"{'hot': False, 'unreviewed_annotations': 0}",Llegué Por San Antonio De Los Baños,Llegué Por San Antonio De Los Baños,https://genius.com/Silvio-rodriguez-llegue-por...
877,1,/songs/1809491,Ismael Serrano,Nana Para Un Niño Indigena by Ismael Serrano,https://assets.genius.com/images/default_cover...,https://assets.genius.com/images/default_cover...,1809491,"\n\nDuerme mi cielo\nMi niño eterno, dueño del...",1549345,complete,/Ismael-serrano-nana-para-un-nino-indigena-lyrics,"{'api_path': '/artists/55590', 'header_image_u...",NaN,https://assets.genius.com/images/default_cover...,"{'hot': False, 'unreviewed_annotations': 0}",Nana Para Un Niño Indigena,Nana Para Un Niño Indigena,https://genius.com/Ismael-serrano-nana-para-un...
1546,1,/songs/1264270,Silvio Rodríguez,Tiempo de ser fantasma by Silvio Rodríguez,https://images.genius.com/e1d03e9cdf1ac152dd62...,https://images.genius.com/e1d03e9cdf1ac152dd62...,1264270,\n\nMientras la noche viajaba\nLentamente haci...,1549345,complete,/Silvio-rodriguez-tiempo-de-ser-fantasma-lyrics,"{'api_path': '/artists/154366', 'header_image_...",NaN,https://images.genius.com/e1d03e9cdf1ac152dd62...,"{'hot': False, 'unreviewed_annotations': 0}",Tiempo de ser fantasma,Tiempo de ser fantasma,https://genius.com/Silvio-rodriguez-tiempo-de-...
2106,1,/songs/1402566,Luis Eduardo Aute,Las huellas del crimen by Luis Eduardo Aute,https://images.genius.com/6589ca03dd7f8e19e5a7...,https://images.genius.com/6589ca03dd7f8e19e5a7...,1402566,"\n\nTe delata\nTu mirada huidiza, distante\nAu...",1549345,complete,/Luis-eduardo-aute-las-huellas-del-crimen-lyrics,"{'api_path': '/artists/353493', 'header_image_...",NaN,https://images.genius.com/6589ca03dd7f8e19e5a7...,"{'hot': False, 'unreviewed_annotations': 0}",Las huellas del crimen,Las huellas del crimen,https://genius.com/Luis-eduardo-aute-las-huell...
2443,1,/songs/1201723,Ch

## Drop all columns i don't need and clean new lines

In [94]:
df = df[['artist', 'title', 'lyrics']]
df['lyrics'] = df['lyrics'].str.replace('\n', ' ')
df.sample(10)

,artist,title,lyrics
2772,Andrés Calamaro,No me nombres,Ahora que parece que para siempre No dura ta...
2814,Andrés Calamaro,Plástico Fino,Buen día primeras luces del día Buen día a p...
1209,Mercedes Sosa,Quena,Del antiguo Cuzco cruzó hasta Arequipa Y el ...
2179,Luis Eduardo Aute,¡Que Necesidad!,Hay veces que aún me pregunto Y sé que no te...
1734,Joaquín Sabina,"Oiga, Doctor","Oiga, doctor Devuélvame mi depresión ¿no ve ..."
2119,Luis Eduardo Aute,Los ojos,Ojos que el tiempo ha borrado su brillo Ojos...
1106,Mercedes Sosa,Indulto,Aunque me quede ciego y sin pensamientos Hab...
2063,Luis Eduardo Aute,Flor de Ceniza,"Dime, amor, cómo se atiza Una brasa en su pe..."
1848,Fabiana Cantilo,Mago En Prosa,Y mi amor no te sirve de nada En tu viaje me...
1075,Mercedes Sosa,"El Tiempo, el Implacable, el que Pasó","El tiempo, el implacable El que paso Siempre..."


In [95]:
df.to_csv('data/related_lyrics.csv', sep=';',index=False)

In [96]:
df

,artist,title,lyrics
0,Kevin Johansen,Acción!,La escena empieza con un hombre que se escon...
1,Kevin Johansen,All I Wanna Do Is You,"Waste my time just thinking of, drinking of ..."
2,Kevin Johansen,"Alta, fea y linda","Era alta, fea y linda Todo en contra y todo ..."
3,Kevin Johansen,Amistad de borrachera,Amistad de borrachera se llama ésta Amistad ...
4,Kevin Johansen,Amor finito,Cuando el amanecer dejo de amenazar Nació la...
5,Kevin Johansen,Amor finito - en vivo,Cuando el amanecer dejo de amenazar Nació la...
6,Kevin Johansen,Anoche soñé contigo,Anoche soñé contigo y no estaba durmiendo To...
7,Kevin Johansen,Anoche soñe contigo - en vivo,Anoche soñé contigo y no estaba durmiendo To...
8,Kevin Johansen,Apocalypso,"Si se mueve todo el piso, ponte liso Baila e..."
9,Kevin Johansen,"Atahualpa, You Funky!","Atahualpa Yupanqui! Atahualpa, you Funky!"
